# Objet : Test opendata des données IRVE

## Objectif

- valider sur des cas réels l'outil de traitement des "listes indexées"
- identifier les apports que pourraient avoir ce type d'outil

## Résultats
- l'outil fonctionne correctement (pas d'erreur identifiées, les temps de réponse restent néanmoins à améliorer).
- l'analyse identifie des données anormales qui seraient à corriger (ou à expliquer) -> cf exemple dans la dernière cellule
- le gain en taille de fichier varie de 50% (format texte non optimisé) à 80% (taille divisée par 5 !) dans un format optimisé et binaire. Ce niveau d'optimisation est tout à fait notable.
- l'analyse permet de (re)trouver la logique de dépendance entre les colonnes

## Usages possibles 
- les indicateurs utilisés permettent de qualifier la nature du fichier csv (niveau de duplication des données) ainsi que le niveau de codage des données). Ce type d'indicateur est déployable simplement.
- l'usage de format de données moins gourmand (sans dégradation des données) peut également être intéressant à déployer
- la qualité des données peut faire l'objet d'un indicateur qui mesure l'écart entre les données attendues et les données existantes. Il pourrait également être utilisé pour vérfier des règles fixées au préalable.

## Autres points
- capacité ok de l'objet Iindexset à traiter des structures de données importantes (50 colonnes)
- indicateurs et fonctions de conversion pertinentes
- temps de réponse de la fonction 'coupling' à regarder
- tests complémentaires à effectuer sur d'autres jeux de données
- représentation graphique de la structure des données à regarder
- indicateur qualité à regarder

données utilisées : https://static.data.gouv.fr/resources/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/20220629-080611/consolidation-etalab-schema-irve-v-2.0.2-20220628.csv

------
## Initialisation
- lecture du fichier issu de l'api (quelques lignes incohérentes au niveau csv ont été éliminées au préalable)

In [32]:
from pprint import pprint
from collections import Counter
from time import time
import csv
import os
os.chdir('C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/python ESstandard/ES')
from iindex import Iindex, util
from iindexset import Iindexset

chemin = 'C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/python ESstandard/validation/irve/'

file = chemin + 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre.csv'

print('file size : ', os.stat(file).st_size)

file size :  7467244


In [33]:
t0 = time()
with open(file, newline='', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=';')
    names = next(reader)
    data = []
    for row in reader: data.append(row)
data2 = util.list(list(zip(*data)))
print('data', time()-t0)

data 0.5849692821502686


----
## initialisation de l'objet Iindexset
- l'initialisation pourrait être automatisée à partir du fichier csv
- identification de 64 775 valeurs différentes sur un total de 11 163 x 49 valeurs ("taux d'unicité" de 12%)
- la taille minimale serait de 1,4 Mo (données csv "quotées") pour un maximum de 9,6 Mo (données csv "quotées")

In [34]:
t0=time()
idxs = Iindexset.Iext(data2, names, fast=True)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs), len(idxs.idxlen), sum(idxs.idxlen), time()-t0)
t0=time()
fullsize = len(idxs.to_obj(encoded=True, fullcodec=True))
print('fullsize', fullsize, time()-t0)
t0=time()
minsize = len(idxs.to_obj(encoded=True, defaultcodec=True, fullcodec=True))
print('minsize', minsize, time()-t0)

idxs (len, lenlidx, sumcodec) :  11163 49 64775 1.9536161422729492
fullsize 9615178 13.746973037719727
minsize 1388222 1.8562648296356201


----
## format non optimisé
- le "taux d'unicité" reste à 12% (pas de modification des index)
- le "taux de codage" est de 30% (remplacement des données dupliquées par un entier)
- le gain de taille de fichier par rapport à un fichier "quoté" est de 61%
- l'analyse de la structure montre que les données sont principalement du type "linked" (non ou peu structuré)
- quelques colonnes sont de type "derived". Par exemple les index longitude(43) et latitude(44) sont bien dérivés de l'index coordonneesXY(13)
- le taux de couplage ("linkrate") pour chacun des index est très proche de 0, ce qui signifie que les données devraient être de type "derived" (lien de dépendance par exemple comme entre les trimestres et les mois)

In [35]:
t0=time()
defaultsize = len(idxs.to_obj(encoded=True, defaultcodec=True))
print('defaultsize', defaultsize, time()-t0)
print('indicator default : ', idxs.indicator(fullsize, defaultsize))
pprint(idxs.indexinfos(keys=['num', 'name', 'lencodec', 'parent', 'typecoupl'], base=True), width=120)
pprint(idxs.indexinfos(keys=['linkrate']))

defaultsize 3725037 2.2882091999053955
indicator default :  {'unique values': 64824, 'unicity level': 0.116, 'mean size': 5.287, 'object lightness': 0.307, 'gain': 0.613}
[{'lencodec': 481, 'name': '\ufeffnom_amenageur', 'num': 0, 'parent': 9, 'typecoupl': 'linked'},
 {'lencodec': 670, 'name': 'siren_amenageur', 'num': 1, 'parent': 9, 'typecoupl': 'linked'},
 {'lencodec': 388, 'name': 'contact_amenageur', 'num': 2, 'parent': 9, 'typecoupl': 'linked'},
 {'lencodec': 152, 'name': 'nom_operateur', 'num': 3, 'parent': 39, 'typecoupl': 'linked'},
 {'lencodec': 159, 'name': 'contact_operateur', 'num': 4, 'parent': 39, 'typecoupl': 'linked'},
 {'lencodec': 215, 'name': 'telephone_operateur', 'num': 5, 'parent': 13, 'typecoupl': 'linked'},
 {'lencodec': 764, 'name': 'nom_enseigne', 'num': 6, 'parent': 9, 'typecoupl': 'linked'},
 {'lencodec': 7060, 'name': 'id_station_itinerance', 'num': 7, 'parent': 15, 'typecoupl': 'linked'},
 {'lencodec': 3838, 'name': 'id_station_local', 'num': 8, 'parent':

----
## couplage
- transformation des index qui devraient être couplés en index couplés (tous les index sont bien maintenant de type "derived")


In [36]:
t0=time()
infos = idxs.coupling(fast=True)
print('coupling', time()-t0)
pprint(idxs.indexinfos(keys=['num', 'name', 'parent', 'typecoupl']), width=120)
pprint(idxs.indexinfos(keys=['num', 'name', 'parent', 'typecoupl']), width=120)

coupling 55.95050048828125
[{'name': '\ufeffnom_amenageur', 'num': 0, 'parent': 9, 'typecoupl': 'derived'},
 {'name': 'siren_amenageur', 'num': 1, 'parent': 9, 'typecoupl': 'derived'},
 {'name': 'contact_amenageur', 'num': 2, 'parent': 9, 'typecoupl': 'derived'},
 {'name': 'nom_operateur', 'num': 3, 'parent': 39, 'typecoupl': 'derived'},
 {'name': 'contact_operateur', 'num': 4, 'parent': 39, 'typecoupl': 'derived'},
 {'name': 'telephone_operateur', 'num': 5, 'parent': 13, 'typecoupl': 'derived'},
 {'name': 'nom_enseigne', 'num': 6, 'parent': 15, 'typecoupl': 'derived'},
 {'name': 'id_station_itinerance', 'num': 7, 'parent': 15, 'typecoupl': 'derived'},
 {'name': 'id_station_local', 'num': 8, 'parent': 15, 'typecoupl': 'derived'},
 {'name': 'nom_station', 'num': 9, 'parent': 15, 'typecoupl': 'derived'},
 {'name': 'implantation_station', 'num': 10, 'parent': 8, 'typecoupl': 'derived'},
 {'name': 'adresse_station', 'num': 11, 'parent': 9, 'typecoupl': 'derived'},
 {'name': 'code_insee_com

----
## Format optimisé
- le "taux d'unicité" se dégrade légèrement (passage de 11,6% à 12,1%) par l'ajout d'index supplémentaires
- le "taux de codage" par contre passe de 30% à 16% de par l'optimisation 
- le gain de taille de fichier par rapport à un fichier "quoté" est maintenant de 74%
- l'utilisation d'un format binaire (codage CBOR pour Concise Binary Object Representation RFC 8949) permet d'améliorer encore le gain de taille de fichier (82%)

In [37]:
t0=time()
optimizesize = len(idxs.to_obj(indexinfos=infos, encoded=True))
print('optimizesize ', optimizesize, time()-t0)
print('indicator optimize : ', idxs.indicator(fullsize, optimizesize))
t0=time()
js = idxs.to_obj(indexinfos=infos, encoded=True, encode_format='cbor')
cborsize = len(js)
print('cborsize', cborsize, time()-t0)
print('indicator cbor : ', idxs.indicator(fullsize, cborsize))

optimizesize  2495885 10.116098403930664
indicator optimize :  {'unique values': 67357, 'unicity level': 0.121, 'mean size': 2.721, 'object lightness': 0.158, 'gain': 0.74}
cborsize 1717876 10.97364854812622
indicator cbor :  {'unique values': 67357, 'unicity level': 0.121, 'mean size': 1.136, 'object lightness': 0.066, 'gain': 0.821}


----
## Intégrité
- la transformation inverse des données binaires permet de vérifier qu'on retombe bien sur les mêmes données (pas de dégradation)

In [38]:
t0=time()
idxs2 = Iindexset.from_obj(js)
print('fromcbor', len(idxs2), time()-t0)
t0=time()
verif = idxs2 == idxs
print('controle égalité :', verif, time()-t0)


fromcbor 11163 3.348214864730835
controle égalité : True 0.24997305870056152


----
## Exemple de données anormales
- l'index coordonneesXY(13) est lié à l'index nom_station(9) avec un taux de couplage très faible, par ailleurs, le nombre de valeurs de ces deux index sont très proches (4503 nom_station pour 4459 coordonneesXY), ce qui signifie que dans la majorité des cas, on associe de façon unique une station et une position
- les exemples ci-dessous montrent les écarts les plus important :
    - la position [1.106329, 49.474202] est associée à 10 stations
    - la station Camping Arinella est associée à 5 positions

In [41]:
print('Couplage entre ', idxs[13].name, ' et ', idxs[9].name, ' : ', idxs[13].couplinginfos(idxs[9])['typecoupl'])
infosdefault = idxs[9].couplinginfos(idxs[13], default=True)
print('Ecart : ', infosdefault['disttomin'], 'positions sur ', infosdefault['distmin'], '\n') # moins de 1%
nom_station = idxs[9].tostdcodec(full=False)
coordonneesXY = idxs[13].tostdcodec(full=False) 
coordonneesXY.coupling(nom_station)
c = Counter(coordonneesXY.codec).most_common(5)
print('les 5 positions avec le plus de stations: ', c, '\n')
print('liste des stations associées à la position', c[0][0], ' : ', 
      [nom_station[i] for i in coordonneesXY.recordfromvalue(c[0][0])], '\n')
coordonneesXY = idxs[13].tostdcodec(full=False)
nom_station.coupling(coordonneesXY)
c = Counter(nom_station.codec).most_common(5)
print('les 5 stations avec le plus de positions: ', c, '\n')
print('liste des positions associées à la station', c[0][0], ' : ', 
      [coordonneesXY[i] for i in nom_station.recordfromvalue(c[0][0])])

Couplage entre  coordonneesXY  et  nom_station  :  derived
Ecart :  44 positions sur  4503 

les 5 positions avec le plus de stations:  [('[1.106329, 49.474202]', 10), ('[1.313367, 49.137233]', 6), ('[3.080477, 50.675889]', 6), ('[2.523685, 48.9908]', 5), ('[3.1557445, 50.5161745]', 4)] 

liste des stations associées à la position [1.106329, 49.474202]  :  ['SCH01', 'SCH10', 'SCH09', 'SCH07', 'SCH06', 'SCH05', 'SCH04', 'SCH03', 'SCH02', 'SCH08'] 

les 5 stations avec le plus de positions:  [('Camping Arinella', 5), ('GUERET', 4), ('Noues de Sienne, Le Bourg', 3), ('PONTORSON - aire de covoiturage', 2), ('Beaumont Saint-Cyr', 2)] 

liste des positions associées à la station Camping Arinella  :  ['[9.445075, 41.995246]', '[9.445074, 41.995246]', '[9.445073, 41.995246]', '[9.445072, 41.995246]', '[9.445071, 41.995246]']
